Multihead attention 

In [2]:
import torch
import torch.nn as nn 

C:\Users\radcl\AppData\Roaming\Python\Python313\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [3]:
inputs = torch.nn.Embedding(4,8)

In [4]:
inputs = inputs.weight.data

In [5]:
inputs.shape

torch.Size([4, 8])

In [6]:
#1. Creating a batch of inputs
batches = torch.stack((inputs,inputs), dim = 0)

In [7]:
# output dimension
d_out = 12
d_in = 4
num_heads = 4
assert( d_out % num_heads ==0), "Error: num_heads must be divisible by d_out, try again"


b, num_tokens, d_in = batches.shape
print(batches)
print("Batches Shape:", batches.shape)

    

# make sure d_out is always divisible by num_heads, otherwise the matrix multiplication does not work 

tensor([[[ 0.1308, -0.6282, -0.5783,  2.4659, -1.0366, -2.0084,  0.2694,
          -0.5123],
         [-1.5517,  0.2296, -0.4225,  0.5770,  0.0938,  1.3966, -1.2605,
           0.4180],
         [-1.3177,  1.0482,  1.7603,  1.6768, -0.6987,  0.8940, -0.2337,
          -1.6383],
         [ 0.6325, -0.2742, -0.2992,  0.2525, -1.1736, -0.5618,  1.2594,
          -1.8560]],

        [[ 0.1308, -0.6282, -0.5783,  2.4659, -1.0366, -2.0084,  0.2694,
          -0.5123],
         [-1.5517,  0.2296, -0.4225,  0.5770,  0.0938,  1.3966, -1.2605,
           0.4180],
         [-1.3177,  1.0482,  1.7603,  1.6768, -0.6987,  0.8940, -0.2337,
          -1.6383],
         [ 0.6325, -0.2742, -0.2992,  0.2525, -1.1736, -0.5618,  1.2594,
          -1.8560]]])
Batches Shape: torch.Size([2, 4, 8])


In [ ]:
#In a sequence of cells, go step-by-step through the same calculation of
# queries and attention scores that the MultiHeadAttention class does.
# At each step, print the tensors and their shapes to see why reshaping is necessary.

In [69]:
#setup parameter
W_query = nn.Linear(d_in, d_out, bias = False)
W_key = nn.Linear(d_in, d_out, bias = False)
W_value = nn.Linear(d_in, d_out, bias = False)

print(W_query,W_key,W_value)

keys = W_key(batches)
queries = W_query(batches)
values = W_value(batches)

print("Queries:",queries,queries.shape)
print("Keys:",keys,keys.shape)
print("Values:",values, values.shape)



Linear(in_features=8, out_features=12, bias=False) Linear(in_features=8, out_features=12, bias=False) Linear(in_features=8, out_features=12, bias=False)
Queries: tensor([[[ 0.1164, -0.6811,  0.1669,  0.0357, -0.1414,  0.2240,  0.1928,
          -0.5178, -0.4856, -0.8122, -0.0802, -0.2613],
         [-0.7732,  0.0645, -0.6681, -0.1394, -0.0730, -0.1375,  0.5155,
           0.2867, -0.6223,  0.5112, -0.3628,  0.0926],
         [ 0.2868, -0.3071,  0.9084,  0.1792,  0.0425, -0.3077, -0.8131,
           0.7887,  1.0414, -0.3755, -0.6892,  0.3011],
         [-0.9935,  0.1487, -0.6568,  0.4441,  0.5104, -0.6038, -0.1148,
           1.0410,  0.0992,  0.2737, -0.4814,  0.8268]],

        [[ 0.1164, -0.6811,  0.1669,  0.0357, -0.1414,  0.2240,  0.1928,
          -0.5178, -0.4856, -0.8122, -0.0802, -0.2613],
         [-0.7732,  0.0645, -0.6681, -0.1394, -0.0730, -0.1375,  0.5155,
           0.2867, -0.6223,  0.5112, -0.3628,  0.0926],
         [ 0.2868, -0.3071,  0.9084,  0.1792,  0.0425, -0.3077

In [ ]:
queries = queries.view(b,num_tokens, num_heads, d_out // num_heads)
keys = keys.view(b,num_tokens, num_heads, d_out // num_heads)
values = values.view(b,num_tokens, num_heads, d_out // num_heads)




print("Queries reshaped:",queries,"Q shape:",queries.shape)
print("Keys reshaped:",keys,"K shape:",keys.shape)
print("Values reshaped:",values,"V shape:",values.shape) 

Queries reshaped: tensor([[[[ 0.1164, -0.6811,  0.1669],
          [-0.7732,  0.0645, -0.6681],
          [ 0.2868, -0.3071,  0.9084],
          [-0.9935,  0.1487, -0.6568]],

         [[ 0.0357, -0.1414,  0.2240],
          [-0.1394, -0.0730, -0.1375],
          [ 0.1792,  0.0425, -0.3077],
          [ 0.4441,  0.5104, -0.6038]],

         [[ 0.1928, -0.5178, -0.4856],
          [ 0.5155,  0.2867, -0.6223],
          [-0.8131,  0.7887,  1.0414],
          [-0.1148,  1.0410,  0.0992]],

         [[-0.8122, -0.0802, -0.2613],
          [ 0.5112, -0.3628,  0.0926],
          [-0.3755, -0.6892,  0.3011],
          [ 0.2737, -0.4814,  0.8268]]],


        [[[ 0.1164, -0.6811,  0.1669],
          [-0.7732,  0.0645, -0.6681],
          [ 0.2868, -0.3071,  0.9084],
          [-0.9935,  0.1487, -0.6568]],

         [[ 0.0357, -0.1414,  0.2240],
          [-0.1394, -0.0730, -0.1375],
          [ 0.1792,  0.0425, -0.3077],
          [ 0.4441,  0.5104, -0.6038]],

         [[ 0.1928, -0.5178, -0.

In [81]:
# Transpose by head
queries = queries.transpose(1,2)
keys = keys.transpose(1,2)
values = values.transpose(1,2)

print("queries transposed:", queries, "Shape:", queries.shape)
print("Keys transposed:", keys, "Shape:", keys.shape)
print("values transposed:", values, "Shape:", values.shape)



queries transposed: tensor([[[[ 0.1164, -0.6811,  0.1669],
          [ 0.0357, -0.1414,  0.2240],
          [ 0.1928, -0.5178, -0.4856],
          [-0.8122, -0.0802, -0.2613]],

         [[-0.7732,  0.0645, -0.6681],
          [-0.1394, -0.0730, -0.1375],
          [ 0.5155,  0.2867, -0.6223],
          [ 0.5112, -0.3628,  0.0926]],

         [[ 0.2868, -0.3071,  0.9084],
          [ 0.1792,  0.0425, -0.3077],
          [-0.8131,  0.7887,  1.0414],
          [-0.3755, -0.6892,  0.3011]],

         [[-0.9935,  0.1487, -0.6568],
          [ 0.4441,  0.5104, -0.6038],
          [-0.1148,  1.0410,  0.0992],
          [ 0.2737, -0.4814,  0.8268]]],


        [[[ 0.1164, -0.6811,  0.1669],
          [ 0.0357, -0.1414,  0.2240],
          [ 0.1928, -0.5178, -0.4856],
          [-0.8122, -0.0802, -0.2613]],

         [[-0.7732,  0.0645, -0.6681],
          [-0.1394, -0.0730, -0.1375],
          [ 0.5155,  0.2867, -0.6223],
          [ 0.5112, -0.3628,  0.0926]],

         [[ 0.2868, -0.3071,  

In [82]:
attention_scores = queries @ keys.transpose(2,3)
print("Attention Scores:", attention_scores)
print("Attention Score shape:", attention_scores.shape)

Attention Scores: tensor([[[[ 0.1842,  0.2268,  0.4314, -0.3223],
          [ 0.0461, -0.0402,  0.0805, -0.1389],
          [ 0.2044,  0.4954,  0.3333,  0.0161],
          [-0.5024, -0.0750,  0.2026, -0.0922]],

         [[-0.1101,  0.0674,  0.4677, -0.3244],
          [-0.0037,  0.0497,  0.1174, -0.0652],
          [ 0.4349, -0.0672,  0.1449, -0.0489],
          [ 0.2405,  0.1285, -0.0370,  0.1158]],

         [[ 0.1249, -0.0035,  0.1753, -0.9563],
          [-0.3803, -0.0088, -0.1132,  0.0212],
          [ 2.0497, -0.1870,  0.9492,  0.2277],
          [ 0.2188,  0.1909, -0.2606,  0.1416]],

         [[-0.0643, -0.2665,  1.0756, -0.2845],
          [-0.0191,  0.8494,  0.2290, -0.2609],
          [ 0.5532, -0.2796, -0.4791,  0.2532],
          [ 0.0399, -0.4498, -0.7968,  0.3531]]],


        [[[ 0.1842,  0.2268,  0.4314, -0.3223],
          [ 0.0461, -0.0402,  0.0805, -0.1389],
          [ 0.2044,  0.4954,  0.3333,  0.0161],
          [-0.5024, -0.0750,  0.2026, -0.0922]],

         [

In [83]:

mask_bool = (torch.tril(torch.ones(num_tokens,num_tokens))==0)
attention_scores.masked_fill_(mask_bool,-torch.inf)


tensor([[[[ 0.1842,    -inf,    -inf,    -inf],
          [ 0.0461, -0.0402,    -inf,    -inf],
          [ 0.2044,  0.4954,  0.3333,    -inf],
          [-0.5024, -0.0750,  0.2026, -0.0922]],

         [[-0.1101,    -inf,    -inf,    -inf],
          [-0.0037,  0.0497,    -inf,    -inf],
          [ 0.4349, -0.0672,  0.1449,    -inf],
          [ 0.2405,  0.1285, -0.0370,  0.1158]],

         [[ 0.1249,    -inf,    -inf,    -inf],
          [-0.3803, -0.0088,    -inf,    -inf],
          [ 2.0497, -0.1870,  0.9492,    -inf],
          [ 0.2188,  0.1909, -0.2606,  0.1416]],

         [[-0.0643,    -inf,    -inf,    -inf],
          [-0.0191,  0.8494,    -inf,    -inf],
          [ 0.5532, -0.2796, -0.4791,    -inf],
          [ 0.0399, -0.4498, -0.7968,  0.3531]]],


        [[[ 0.1842,    -inf,    -inf,    -inf],
          [ 0.0461, -0.0402,    -inf,    -inf],
          [ 0.2044,  0.4954,  0.3333,    -inf],
          [-0.5024, -0.0750,  0.2026, -0.0922]],

         [[-0.1101,    -inf,

In [84]:


attention_weights = torch.softmax(attention_scores / keys.shape[-1]**0.5, dim=-1)
attention_weights = torch.dropout(attention_weights, p=0, train=True)

attention_weights,attention_weights.shape

(tensor([[[[1.0000, 0.0000, 0.0000, 0.0000],
           [0.5125, 0.4875, 0.0000, 0.0000],
           [0.3067, 0.3628, 0.3304, 0.0000],
           [0.1980, 0.2535, 0.2975, 0.2510]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.4923, 0.5077, 0.0000, 0.0000],
           [0.3855, 0.2885, 0.3260, 0.0000],
           [0.2688, 0.2520, 0.2290, 0.2501]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.4466, 0.5534, 0.0000, 0.0000],
           [0.5541, 0.1523, 0.2935, 0.0000],
           [0.2704, 0.2660, 0.2050, 0.2586]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.3772, 0.6228, 0.0000, 0.0000],
           [0.4610, 0.2850, 0.2540, 0.0000],
           [0.2802, 0.2112, 0.1729, 0.3357]]],
 
 
         [[[1.0000, 0.0000, 0.0000, 0.0000],
           [0.5125, 0.4875, 0.0000, 0.0000],
           [0.3067, 0.3628, 0.3304, 0.0000],
           [0.1980, 0.2535, 0.2975, 0.2510]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.4923, 0.5077, 0.0000, 0

In [87]:
        
context_vec = (attention_weights @ values.transpose(1,2)) 
context_vec

tensor([[[[ 0.0490, -0.9331,  0.7512],
          [ 0.1413, -0.4448,  0.2627],
          [-0.2899, -0.5274, -0.0349],
          [-0.3392, -0.5007, -0.1556]],

         [[ 0.3188, -0.6036,  0.4032],
          [-0.0381, -0.2422,  0.1890],
          [-0.0294, -0.6630,  0.1342],
          [-0.2495, -0.4849, -0.0359]],

         [[ 0.1633, -0.2368, -0.4353],
          [-0.2652, -0.5139,  0.1809],
          [ 0.2068,  0.0395, -0.2457],
          [-0.1112, -0.2590,  0.2009]],

         [[ 0.1264, -0.3258,  0.4194],
          [ 0.1818,  0.0610, -0.2628],
          [-0.0775, -0.3540,  0.0866],
          [-0.3472, -0.1039, -0.2532]]],


        [[[ 0.0490, -0.9331,  0.7512],
          [ 0.1413, -0.4448,  0.2627],
          [-0.2899, -0.5274, -0.0349],
          [-0.3392, -0.5007, -0.1556]],

         [[ 0.3188, -0.6036,  0.4032],
          [-0.0381, -0.2422,  0.1890],
          [-0.0294, -0.6630,  0.1342],
          [-0.2495, -0.4849, -0.0359]],

         [[ 0.1633, -0.2368, -0.4353],
          [

In [88]:
 
context_vec = context_vec.contiguous().view(b,num_tokens,d_out)
context_vec, context_vec.shape

        

(tensor([[[ 0.0490, -0.9331,  0.7512,  0.1413, -0.4448,  0.2627, -0.2899,
           -0.5274, -0.0349, -0.3392, -0.5007, -0.1556],
          [ 0.3188, -0.6036,  0.4032, -0.0381, -0.2422,  0.1890, -0.0294,
           -0.6630,  0.1342, -0.2495, -0.4849, -0.0359],
          [ 0.1633, -0.2368, -0.4353, -0.2652, -0.5139,  0.1809,  0.2068,
            0.0395, -0.2457, -0.1112, -0.2590,  0.2009],
          [ 0.1264, -0.3258,  0.4194,  0.1818,  0.0610, -0.2628, -0.0775,
           -0.3540,  0.0866, -0.3472, -0.1039, -0.2532]],
 
         [[ 0.0490, -0.9331,  0.7512,  0.1413, -0.4448,  0.2627, -0.2899,
           -0.5274, -0.0349, -0.3392, -0.5007, -0.1556],
          [ 0.3188, -0.6036,  0.4032, -0.0381, -0.2422,  0.1890, -0.0294,
           -0.6630,  0.1342, -0.2495, -0.4849, -0.0359],
          [ 0.1633, -0.2368, -0.4353, -0.2652, -0.5139,  0.1809,  0.2068,
            0.0395, -0.2457, -0.1112, -0.2590,  0.2009],
          [ 0.1264, -0.3258,  0.4194,  0.1818,  0.0610, -0.2628, -0.0775,
      